# Quantum dynamics resource estimation

In this Q# notebook we demonstrate resource estimation for quantum dynamics,
specifically the simulation of an Ising model Hamiltonian on an $N \times N$ 2D
lattice using a *fourth-order Trotter Suzuki product formula* assuming a 2D
qubit architecture with nearest-neighbor connectivity.

First, we connect to the Azure quantum service and load the necessary packages.

In [ ]:
from azure.quantum import Workspace
from azure.quantum.target.microsoft import MicrosoftEstimator, QubitParams, QECScheme
import qsharp

In [ ]:
workspace = Workspace (
    resource_id = "",
    location = ""
)

In [ ]:
qsharp.packages.add("Microsoft.Quantum.Numerics")

## Background: 2D Ising model

The Ising model is a mathematical model of ferromagnetism in a lattice (in our case a 2D square lattice) with two kinds of terms in the Hamiltonian: (i) an interaction term between adjacent sites and (ii) an external magnetic field acting at each site. For our purposes, we consider a simplified version of the model where the interaction terms have the same strength and the external field strength is the same at each site.
Formally, the Ising model Hamiltonian on an $N \times N$ lattice we consider is formulated as:

$$
H = \underbrace{-J \sum_{i, j} Z_i Z_j}_{B} + \underbrace{g \sum_j X_j}_{A}
$$
where $J$ is the interaction strength, $g$ is external field strength.

The time evolution $e^{-iHt}$ for the Hamiltonian is simulated with the fourth-order product formula so that any errors in simulation are sufficiently small. Essentially, this is done by simulating the evolution for small slices of time $\Delta$ and repeating this for `nSteps` $= t/\Delta$ to obtain the full time evolution. The Trotter-Suzuki formula for higher orders can be recursively defined using a *fractal decomposition* as discussed in Section 3 of [[Hatanao and Suziki's survey](https://link.springer.com/chapter/10.1007/11526216_2)]. Then the fourth order formula $U_4(\Delta)$ can be constructed using the second-order one $U_2(\Delta)$ as follows.
$$
\begin{aligned}
U_2(\Delta) & = e^{-iA\Delta/2} e^{-iB\Delta} e^{-iA\Delta/2}; \\
U_4(\Delta) & = U_2(p\Delta)U_2(p\Delta)U_2((1 - 4p)\Delta)U_2(p\Delta)U_2(p\Delta); \\
p & = (4 - 4^{1/3})^{-1}.
\end{aligned}
$$

For the rest of the notebook, we will present the code that computes the time evolution in a step by step fashion.

## Implementation

### Helper functions

We will allocate all qubits in the 2D lattice in a one-dimensional array.  The function `GetQubitIndex` converts a qubit identified on a 2D lattice by `(row, col)` to an index in that array. We assume a snake-like order on the 2D lattice i.e., the numbering goes left-to-right on even rows and right-to-left on odd rows.

In [ ]:
%%qsharp
function GetQubitIndex(row : Int, col : Int, n : Int) : Int {
    return row % 2 == 0             // if row is even,
        ? col + n * row             // move from left to right,
        | (n - 1 - col) + n * row;  // otherwise from right to left.
}

Note that expanding $U_4(\Delta)$ to express it in terms of $A, B$ gives:
$$
U_4(\Delta) = e^{-iAp\Delta/2} e^{-iBp\Delta} e^{-iAp\Delta} e^{-iBp\Delta} e^{-iA(1 - 3p)\Delta/2} e^{-iB(1-4p)\Delta} e^{-iA(1 - 3p)\Delta/2} e^{-iBp\Delta} e^{-iAp\Delta} e^{-iBp\Delta} e^{-iAp\Delta/2}
$$

The above equation with $11$ exponential terms works for one time step. For `nSteps` $> 1$ time steps, some adjacent terms can be merged to give $10t+1$ exponential terms for $e^{-iHt}$.

The function below creates two sequences `seqA` and `seqB` corresponding to the constant factors that will be applied with $A$ and $B$, respectively, in the exponential sequence of the above formula.

In [ ]:
%%qsharp
function SetSequences(len : Int, p : Double, dt : Double, J : Double, g : Double) : (Double[], Double[]) {
    // create two arrays of size `len`
    mutable seqA = [0.0, size=len];
    mutable seqB = [0.0, size=len];

    // pre-compute values according to exponents
    let values = [
        -J * p * dt,
        g * p * dt,
        -J * p * dt,
        g * p * dt,
        -J * (1.0 - 3.0 * p) * dt / 2.0,
        g * (1.0 - 4.0 * p) * dt,
        -J * (1.0 - 3.0 * p) * dt / 2.0,
        g * p * dt,
        -J * p * dt,
        g * p * dt
    ];

    // assign first and last value of `seqA`
    set seqA w/= 0 <- -J * p * dt / 2.0;
    set seqA w/= len - 1 <- -J * p * dt / 2.0;

    // assign other values to `seqA` or `seqB`
    // in an alternating way
    for i in 1..len - 2 {
        if i % 2 == 0 {
            set seqA w/= i <- values[i % 10];
        }
        else {
            set seqB w/= i <- values[i % 10];
        }
    }

    return (seqA, seqB);
}

### Quantum operations

There are two kinds of Pauli exponentials needed for simulating the time evolution of an Ising Model:
- The transverse field $e^{-iX\theta}$ applied to each qubit for an angle $\theta$;
- $e^{-i (Z \otimes Z)\theta}$ applied to neighboring pairs of qubits in the lattice.

The operation below applies $e^{-iX\theta}$ on all qubits in the 2D lattice.

In [ ]:
%%qsharp
operation ApplyAllX(qs : Qubit[], theta : Double) : Unit {
    // This applies `Rx` with an angle of `2.0 * theta` to all qubits in `qs`
    // using partial application
    ApplyToEach(Rx(2.0 * theta, _), qs);
}

The next operation below applies $e^{-i(Z \otimes Z)\theta}$ on overlapping pairs of neighboring qubits. We decompose this term into a single qubit $e^{-iZ\theta}$ term (implemented as an `Rz` rotation) conjugated by `CNOT`s to entangle the neighboring qubits following Section 4.2 of [[Whitfield et al.](https://www.tandfonline.com/doi/abs/10.1080/00268976.2011.552441)].

Observe that unlike the previous case, it is not possible to simultaneously apply all the rotations in one go. For example, while applying the rotation on qubits at $(0, 0)$ and $(0, 1)$, it is not possible to also apply the rotation on qubits $(0, 1)$ and $(0, 2)$. Instead, we try to apply as many rotations as possible. This is broken up as follows:
- in the horizontal (resp. vertical) direction of the 2D lattice as chosen by `dir`,
- consider pairs starting with an even (resp. odd) index as given by `grp`;
- apply the exponential to all such pairs in the lattice.

In [ ]:
%%qsharp
operation ApplyDoubleZ(n : Int, qs : Qubit[], theta : Double, dir : Bool, grp : Bool) : Unit {
    let start = grp ? 0 | 1;    // Choose either odd or even indices based on group number

    for i in 0..n - 1 {
        for j in start..2..n - 2 {    // Iterate through even or odd `j`s based on `grp`
            // rows and cols are interchanged depending on direction
            let (row, col) = dir ? (i, j) | (j, i);

            // Choose first qubit based on row and col
            let ind1 = GetQubitIndex(row, col, n);
            // Choose second qubit in column if direction is horizontal and next qubit in row if direction is vertical
            let ind2 = dir ? GetQubitIndex(row, col + 1, n) | GetQubitIndex(row + 1, col, n);

            within {
                CNOT(qs[ind1], qs[ind2]);
            } apply {
                Rz(2.0 * theta, qs[ind2]);
            }
        }
    }
}

The next operation puts everything together and calls the operations needed to
simulate the Ising model Hamiltonian using a fourth order product formula.
Observe that the `ApplyDoubleZ` operation is called four times for different
choices of direction and starting index to ensure all possible pairs of qubits
are appropriately considered.

The various parameters taken in by the operation correspond to:

- `J`, `g`: parameters by which the Hamiltonian terms are scaled.
- `N`: size of the square lattice.
- `totTime`: the number of Trotter steps.
- `dt` : the step size for the simulation, sometimes denoted as $\Delta$.
- `eps`: the precision for arbitrary rotations.

In [ ]:
%%qsharp
open Microsoft.Quantum.Math;

operation IsingModel2DSim(N : Int, J : Double, g : Double, totTime : Double, dt : Double, eps : Double) : Unit {
    use qs = Qubit[N * N];
    let len = Length(qs);

    let p = 1.0 / (4.0 - PowD(4.0, 1.0 / 3.0));
    let t = Ceiling(totTime / dt);

    let seqLen = 10 * t + 1;

    let (seqA, seqB) = SetSequences(seqLen, p, dt, J, g);

    for i in 0..seqLen - 1 {
        // for even indexes
        if i % 2 == 0 {
            ApplyAllX(qs, seqA[i]);
        } else {
            // iterate through all possible combinations for `dir` and `grp`.
            for (dir, grp) in [(true, true), (true, false), (false, true), (false, false)] {
                ApplyDoubleZ(N, qs, seqB[i], dir, grp);
            }
        }
    }
}

## Running the experiment

Next, we are estimating the physical resource estimates to simulate the Ising
model Hamiltonian for a $10 \times 10$ lattice with $J = g = 1.0$, total time
$20$, step size $0.25$, and `eps` ${}=0.001$. As configurations for the
experiment we use all six pre-defined qubit parameters.  As pre-defined QEC
scheme we are using `surface_code` with gate-based qubit parameters (default),
and `floquet_code` with Majorana based qubit parameters.

In [ ]:
estimator = MicrosoftEstimator(workspace)

labels = ["Gate-based µs, 10⁻³", "Gate-based µs, 10⁻⁴", "Gate-based ns, 10⁻³", "Gate-based ns, 10⁻⁴", "Majorana ns, 10⁻⁴", "Majorana ns, 10⁻⁶"]

params = estimator.make_params(num_items=6)
params.arguments["N"] = 10
params.arguments["J"] = 1.0
params.arguments["g"] = 1.0
params.arguments["totTime"] = 20.0
params.arguments["dt"] = 0.25
params.arguments["eps"] = 0.001
params.items[0].qubit_params.name = QubitParams.GATE_US_E3
params.items[1].qubit_params.name = QubitParams.GATE_US_E4
params.items[2].qubit_params.name = QubitParams.GATE_NS_E3
params.items[3].qubit_params.name = QubitParams.GATE_NS_E4
params.items[4].qubit_params.name = QubitParams.MAJ_NS_E4
params.items[4].qec_scheme.name = QECScheme.FLOQUET_CODE
params.items[5].qubit_params.name = QubitParams.MAJ_NS_E6
params.items[5].qec_scheme.name = QECScheme.FLOQUET_CODE

We are submitting a resource estimation job with all target parameter configurations.

In [ ]:
job = estimator.submit(IsingModel2DSim, input_params=params)
results = job.get_results()

Finally, we are presenting the experimental results using a summary table.

In [ ]:
results.summary_data_frame(labels=labels)

From the results we can observe that a large fraction of physical qubits is used
for the T factories.  To understand why, it's important to remark that the
overall algorithm runtime is determined based on the number of logical
operations (also called logical cycles or logical depth).  The runtime limits
the number of invocations of a single T factory.  The total number of T factory
copies is computed based on the total number of required T states divided by the
number of possible invocations.  Therefore, if the algorithm would run longer, a
T factory can be invoked more often, which may allow to compute all required T
states with less T factory copies.

Since T factory fraction is high, while at the same time the physical runtime is
relatively small, this is a good opportunity for a space-time optimization based
on the logical depth.  We can make an algorithm run longer, by inserting no-op
(no operation or idle) operations.  We do this using the `logical_depth_factor`
constraint. For example, a value of 2 means that the number of cycles should be
twice as much, i.e., one no-op per operation; or, a value of 1.5 means that the
number of cycles is 50% more, i.e., one no-op for every two operations.

Please note that the algorithm runtime may increase by a larger factor than the
`logical_depth_factor`.  This is because no-ops also can incur logical errors,
and therefore the required logical error rate is lower, which in turn may
increase the required code distance, therefore leading a to a longer execution
time of a logical cycle.

In the balanced implementation that is described in the paper, we increase the
logical depth by a factor of 10.  We do this by updating the `params` variable.
All other parameters remain unchanged.

In [ ]:
params.constraints.logical_depth_factor = 10

job = estimator.submit(IsingModel2DSim, input_params=params)
results_balanced = job.get_results()

We print the results for the balanced implementation as a summary.

In [ ]:
results_balanced.summary_data_frame(labels=labels)

Note how the T factory fraction is much smaller now and the number of total
physical qubits decreased as a result.  The logical depth increased exactly by a
factor of 10, whereas the runtime increased by a factor of at least 10, since in
most cases the code distance is higher.

## Next steps

The numbers in the table match the numbers in the paper [Assessing requirements
for scaling quantum computers to real-world impact](https://aka.ms/AQ/RE/Paper).
Feel free to use this table as a starting point for your own experiments.  For
example, you can

* explore how the results change by modifying the operation arguments of the Ising
  model instance
* explore space- and time-trade-offs by changing the value for
  `logical_depth_factor`
* use other or customized qubit parameters